In [ ]:
from utils import list_images, simulate_cytoplasm
from tqdm import tqdm
from cellpose import models, core
import os
from tifffile import imread
from pathlib import Path
from skimage.measure import regionprops_table
import pandas as pd

#Check if notebook has GPU access
if core.use_gpu()==False:
  raise ImportError("No GPU access, change your runtime")

#Load pre-trained Cellpose-SAM and Spotiflow models
model = models.CellposeModel(gpu=True)

In [ ]:
processed_img_path = "./processed_tiffs"
processed_images = list_images(processed_img_path, format="tif")

In [5]:
for image in tqdm(processed_images):

    # Read .tif file
    img = imread(image)

    # Extract metadata
    field_of_view = Path(image).stem.split("f")[1]
    well_id = Path(image).stem.split("f")[0]

    # Create a dictionary containing all image descriptors
    descriptor_dict = {"well_id": well_id, "FOV": field_of_view}

    print(f"Analyzing: Well: {well_id} FOV:{field_of_view}")

    # Predict nuclei labels using CellposeSAM and simulate cytosol using pyclesperanto
    nuclei_labels, flows, styles = model.eval(img[1], niter=1000) # need to check the arguments
    cytoplasm_labels = simulate_cytoplasm(nuclei_labels, dilation_radius=3)

    props_list = []

    # Extract nuclei avg_int
    props = regionprops_table(label_image=nuclei_labels,
                            intensity_image=img[0],
                            properties=["label", "intensity_mean"])

    # Convert to dataframe
    props_df = pd.DataFrame(props)

    # Rename intensity_mean column to indicate the specific image
    props_df.rename(columns={"intensity_mean": f"nuclei_avg_int"}, inplace=True)

    # Append each props_df to props_list
    props_list.append(props_df)


    # Extract cytoplasm avg_int
    props = regionprops_table(label_image=cytoplasm_labels,
                            intensity_image=img[0],
                            properties=["label", "intensity_mean"])

    # Convert to dataframe
    props_df = pd.DataFrame(props)

    # Rename intensity_mean column to indicate the specific image
    props_df.rename(columns={"intensity_mean": f"cytoplasm_avg_int"}, inplace=True)

    # Append each props_df to props_list
    props_list.append(props_df)

    # Initialize the df with the first df in the list
    props_df = props_list[0]
    # Start looping from the second df in the list
    for df in props_list[1:]:
        props_df = props_df.merge(df, on="label")

    # Add each key-value pair from descriptor_dict to props_df at the specified position
    insertion_position = 0
    for key, value in descriptor_dict.items():
        props_df.insert(insertion_position, key, value)
        insertion_position += 1  # Increment position to maintain the order of keys in descriptor_dict

    # Define the .csv path
    csv_path = "avg_int_per_cell_data.csv"

    # Append to the .csv with new data points each round
    props_df.to_csv(csv_path, mode="a", index=False, header=not os.path.isfile(csv_path))



Analyzing: Well: A01 FOV:00
Analyzing: Well: A01 FOV:01
Analyzing: Well: A01 FOV:02
Analyzing: Well: A01 FOV:03
Analyzing: Well: A01 FOV:04
Analyzing: Well: A01 FOV:05
Analyzing: Well: A01 FOV:06
Analyzing: Well: A01 FOV:07
Analyzing: Well: A01 FOV:08
Analyzing: Well: A01 FOV:09
Analyzing: Well: A01 FOV:10
Analyzing: Well: A01 FOV:11
Analyzing: Well: A02 FOV:00
Analyzing: Well: A02 FOV:01
Analyzing: Well: A02 FOV:02
Analyzing: Well: A02 FOV:03
Analyzing: Well: A02 FOV:04
Analyzing: Well: A02 FOV:05
Analyzing: Well: A02 FOV:06
Analyzing: Well: A02 FOV:07
Analyzing: Well: A02 FOV:08
Analyzing: Well: A02 FOV:09
Analyzing: Well: A02 FOV:10
Analyzing: Well: A02 FOV:11
Analyzing: Well: A03 FOV:00
Analyzing: Well: A03 FOV:01
Analyzing: Well: A03 FOV:02
Analyzing: Well: A03 FOV:03
Analyzing: Well: A03 FOV:04
Analyzing: Well: A03 FOV:05
Analyzing: Well: A03 FOV:06
Analyzing: Well: A03 FOV:07
Analyzing: Well: A03 FOV:08
Analyzing: Well: A03 FOV:09
Analyzing: Well: A03 FOV:10
Analyzing: Well: A03